In [3]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [4]:
env = GridworldEnv()

In [21]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    # IMP : initialize the value function to zeros initially, check if the results change with different initialilzations - shouldn't change ideally

    V = np.zeros(env.nS)
    # the V array is not changed in the following iterations, this is imp! :P
    for _ in range(100):
        while True:
        # TODO: Implement!
            threshold = 0

            for s in range(env.nS):
                v = 0

                # here a is just an index to the set of all possible actions from state s, see - http://book.pythontips.com/en/latest/enumerate.html
                # we are going over all the possible actions from state s
                # policy matrix contains the probability of an action being taken from a state
                for a, action_prob in enumerate(policy[s]):
                    # env.P[s][a] has the transition probabilities as given by the environment - here we are using the environment's knowledge in making decisions
                    for prob, next_state, reward, info in env.P[s][a]:
                        # the order of these values is important, don't change the order! - needs lots of debugging
                        v += action_prob * prob *(reward + discount_factor * V[next_state])
                        # bootstrapping - guessing V[next_state] and using it to make current calculation

                threshold = max(threshold, v - V[s])
                V[s] = v
            # stop if the maximum change in value function is less than some threshold
            if threshold < theta:
                break
            
    return np.array(V)

In [18]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [19]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

**IMP:** To ensure that we hit optimal value function, run the while loop of policy_eval function about 100 times, essentially, we keep on iterating until hitting the optimal value function. But most of the times, value function converges much more faster than this, because we don't really need the precise numbers to choose the optimmal policy

In [20]:
# rendering the value function, in gridworld's shape:
print(v.reshape(env.shape))

[[  0.         -13.99765839 -19.99663362 -21.99629468]
 [-13.99765839 -17.99712654 -19.99688008 -19.99691576]
 [-19.99663362 -19.99688008 -17.99736736 -13.99803444]
 [-21.99629468 -19.99691576 -13.99803444   0.        ]]
